In [1]:
import pandas as pd
from konlpy.tag import Okt
from gensim.models import Word2Vec
okt = Okt()
import re
import os
import numpy as np

### Initial Embedding

In [70]:
dl = pd.Series(os.listdir('/opt/notebooks/AA008/ws_fn/데이터수집/data/빅카인즈'))
dlb = dl.apply(lambda x : x.endswith('csv'))
dl = dl[dlb].reset_index(drop=True)

In [71]:
dfs = []
for i in dl:
    tempdf = pd.read_csv('/opt/notebooks/AA008/ws_fn/데이터수집/data/빅카인즈/' + i)
    dfs.append(tempdf)
dfs = pd.concat(dfs,axis=0).reset_index(drop=True)

In [78]:
dfs = dfs[dfs['기준일자'] < 20181231].sample(10000000).reset_index(drop=True)

In [3]:
def cleaner(sent):
    try:
        sents = sent.split('.')
        sents = [re.sub('<[^<]+?>', '', i).strip() for i in sents]
        sents = [re.sub('[!"#$%&\'()*,./:;<=>?@[\\]^_`{|}~]+ *',' ',i) for i in sents]
        sents = [i for i in sents if (len(i) > 0)]
        return sents
    except:
        return []

In [6]:
rep = pd.read_csv('report.csv')
rep = rep[rep['date'] < '2018-12-31'].reset_index(drop=True).apply(lambda x:cleaner(x))

In [80]:
title = dfs['기사제목'].apply(lambda x:cleaner(x))
summ = dfs['기사내용'].apply(lambda x:cleaner(x))

In [81]:
title = title.tolist()
summ = summ.tolist()
rep = rep.tolist()

In [83]:
with open('etf.txt','r') as f:
    lines = f.readlines()

In [84]:
names = [[i] for i in pd.Series(lines).apply(lambda x:x[6:].strip()).tolist()]

In [93]:
wholecorp = summ+ title
wholecorp = wholecorp + names
corp = []
for i in wholecorp:
    if len(i) > 0:
        corp.append(okt.morphs(i[0]))

* Window size 2로 word vector를 훈련하고, dimension size 300에 embedding

In [94]:
dim=300
ws=2
mc = 10

In [95]:
model = Word2Vec(sentences=corp, window=ws, size=dim, min_count=mc)

In [96]:
initembedding = model.wv.vectors
initembedding = np.vstack([np.zeros(dim),initembedding.mean(axis=0),initembedding.mean(axis=0),initembedding.mean(axis=0),initembedding])

In [97]:
wordtokey = dict()
j=4
for i in model.wv.index2word:
    wordtokey[i] = j
    j += 1

In [98]:
wordtokey['<mask>'] = 0
wordtokey['<unk>'] = 1
wordtokey['<sos>'] = 2
wordtokey['<eos>'] = 3

* Embedding 결과 및 관련 Dictionary 저장

In [99]:
np.save('initembedding2.npy',initembedding)

In [100]:
import json
with open('wordtokey2.json','w') as f:
    json.dump(wordtokey, f)

### 리포트 토큰화

In [11]:
report = pd.read_csv('report.csv',index_col=0)
article = pd.read_csv('article.csv',index_col=0)

In [60]:
import json
with open('wordtokey2.json','r') as f:
    wordtokey = json.load(f)

In [55]:
def wordtotoken(word):
    try:
        token = wordtokey[word]
    except: token = wordtokey['<unk>']     
    return token

def tokenizer(sent):
    tl = [wordtotoken(i) for i in sent]
    return tl

In [56]:
report['token'] = report['okt'].apply(lambda x:eval(x)).apply(lambda x:tokenizer(x))

In [57]:
article['tokens'] = article['okt'].apply(lambda x:eval(x)).apply(lambda x:tokenizer(x))

In [14]:
#최종 결과 데이터
report

,date,report,token,okt
0,2010-12-13,POSCO 005490 현대미포조선 010620 투자의견 Hold 및 목표주가 ...,"[4544, 8591, 377, 2732, 1364, 728, 9598, 23, 3...","['POSCO', '005490', '현', '대미', '포', '조선', '010..."
1,2010-12-13,LG 003550 LG 003550 목표주가를 기존 115 000원에서 135...,"[122, 13573, 122, 13573, 114, 73, 158, 1207, 4...","['LG', '003550', 'LG', '003550', '목표', '주가', '..."
2,2010-12-13,현대모비스 012330 현대모비스 012330 목표주가를 325 000 원으로 상...,"[48, 1309, 8541, 48, 1309, 8541, 114, 73, 158,...","['현대', '모비스', '012330', '현대', '모비스', '012330',..."
3,2010-12-14,하이닉스 000660 신한금융지주회사 055550 일회성 요인 감안해도 1분기 실...,"[533, 5032, 103, 12, 39, 3884, 9704, 45, 206, ...","['하이닉스', '000660', '신', '한', '금융', '지주회사', '05..."
4,2010-12-14,대구은행 005270 LG패션 093050 􀁹 11 1Q 기대 이상의 영업이익 ...,"[916, 88, 47687, 122, 619, 14530, 11693, 251, ...","['대구', '은행', '005270', 'LG', '패션', '093050', '..."
...,...,...,...,...
56692,2021-08-12,코스맥스 192820 역시 남다른 체력 코스맥스 192820 역시 남다른 체력 투자...,"[1194, 1602, 9409, 3393, 9123, 5425, 1194, 160...","['코스', '맥스', '192820', '역시', '남다른', '체력', '코스'..."
56693,2021-08-13,오리온 271560 원가 부담 지속 비용 절감 이상의 자구책 필요 카페24 0420...,"[1838, 18371, 2934, 546, 53, 890, 1852, 128, 9...","['오리온', '271560', '원가', '부담', '지속', '비용', '절감'..."
56694,2021-08-13,컴투스 078340 2Q21 어닝쇼크 백년전쟁 부진 크로니클 일정 연 하나금융지주 ...,"[3065, 8931, 17, 136, 629, 3396, 45584, 383, 5...","['컴투스', '078340', '2', 'Q', '21', '어닝쇼크', '백년전..."
56695,2021-08-13,컴투스 078340 뼈아픈 백년전쟁의 부진 코스맥스 192820 중국 온라인 트렌드...,"[3065, 8931, 9710, 11441, 45584, 9, 383, 1194,...","['컴투스', '078340', '뼈', '아픈', '백년전쟁', '의', '부진'..."


In [58]:
report.to_csv('report2.csv')

In [59]:
article.to_csv('article2.csv')